In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [11]:
data = pd.read_csv('adatok_csv.csv')
data.set_index('Date', inplace=True)
data.index = pd.to_datetime(data.index)
data.sort_index(ascending=False, inplace=True)

data = data.loc['2022-01-03':]
#data = data[]

data = data[['SP500', 'MSCI WORLD', 'MXEA', 'MXEF', 'US Agg Bond', 'Global Bond', 'US HY', 'DAX', 'Amazon', 'Apple ', 'Berkshire']]

C:\Users\User\AppData\Local\Temp\ipykernel_21040\2745233372.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data.index = pd.to_datetime(data.index)


In [14]:
# napi log hozamok
daily_returns = np.log(data / data.shift(-1))
# évesített várható hozam
mean_returns = daily_returns.mean() * 250
# évesített kovariancia mátrix
cov_matrix = daily_returns.cov() * 250
std = np.sqrt(np.diagonal(cov_matrix))
num_assets = len(mean_returns)


In [15]:
pd.options.display.float_format = '{:.2f}%'.format

asset_summarum = pd.DataFrame(index = mean_returns.index, columns = ['Várható érték', 'Szórás'])
asset_summarum['Várható érték'] = mean_returns   
asset_summarum['Szórás'] = std
asset_summarum*100

,Várható érték,Szórás
SP500,7.19%,17.27%
MSCI WORLD,-0.93%,32.08%
MXEA,1.50%,15.31%
MXEF,-2.30%,15.94%
US Agg Bond,-1.60%,6.76%
Global Bond,-3.70%,6.72%
US HY,3.29%,5.28%
DAX,8.10%,21.33%
Amazon,7.63%,37.72%
Apple,9.52%,26.99%


In [16]:
pd.options.display.float_format = '{:.2f}'.format


In [17]:
import numpy as np
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import jax

In [ ]:

# Generate example data: Historical returns for 3 assets (A, B, C)
returns = np.random.multivariate_normal(mean=[0.05, 0.06, 0.04], 
                                        cov=[[0.1, 0.02, 0.03], 
                                             [0.02, 0.15, 0.04], 
                                             [0.03, 0.04, 0.12]], 
                                        size=100)

# # Bayesian Model using NumPyro

def model():

    mu = numpyro.sample("mu", dist.Normal(0, 1).expand([3]))  # Prior for expected returns
    cov = numpyro.sample("cov", dist.LKJCholesky(3, concentration=1))  # Covariance structure
    
    # Likelihood
    numpyro.sample("returns_obs", dist.MultivariateNormal(mu, scale_tril=cov), obs=returns)

# Inference using NUTS (No-U-Turn Sampler)
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=1000, num_warmup=1000)
mcmc.run(jax.random.PRNGKey(0))  # Random seed

# Extract Posterior Samples
posterior_samples = mcmc.get_samples()
mu_posterior = jnp.mean(posterior_samples['mu'], axis=0)
cov_posterior = jnp.mean(posterior_samples['cov'], axis=0)

# Convert JAX arrays to NumPy
mu_posterior = np.array(mu_posterior)
cov_posterior = np.array(cov_posterior)

# Portfolio optimization (e.g., maximize Sharpe ratio)
from scipy.optimize import minimize

def sharpe_ratio(weights, mu, cov):
    portfolio_return = np.dot(weights, mu)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    return -portfolio_return / portfolio_volatility  # Negative for minimization

# Constraints: weights sum to 1, no shorting
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = [(0, 1) for _ in range(3)]

# Initial guess (equal weights)
initial_weights = np.array([1/3, 1/3, 1/3])

# Optimize
result = minimize(sharpe_ratio, initial_weights, args=(mu_posterior, cov_posterior), bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal Portfolio Weights:", optimal_weights)


sample: 100%|██████████| 2000/2000 [00:03<00:00, 663.55it/s, 15 steps of size 2.20e-01. acc. prob=0.93] 

Optimal Portfolio Weights: [0.         0.38023401 0.61976599]
